# Processing Dyadic Data 

This notebook processes the data associated with [How inter-state amity and animosity complement migration networks to drive refugee flows: A multi-layer network analysis, 1991–2016](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0245712) into a machine learning ready format.

The data used in that paper is stored across several files. Each file has distinct features; features are organized in individual columns. However, the main issue to overcome is that `year` is included in the columns header. So, for example, a single file might have columns called `trade.2000`, `trade.2001`, `trade.2002`, etc. No column is provided for `year`. This notebook combines these files, cleans the column headers (to remove `year`) and adds a `year` feature.

Additionally, it filters for only dyads that had greater than 1000 refugees from the origin to destination country _in at least one year_. 

To speed up the data processing, multiple cores are used.

In [1]:
import pandas as pd
import re

import numpy as np
import multiprocess as mp
 
cores = mp.cpu_count()-2 #Number of CPU cores on your system
partitions = cores #Define as many partitions as you want

from glob import glob

In [12]:
df1 = pd.read_csv('/Users/brandonrose/Downloads/S1_File/Data_1.csv')
df2 = pd.read_csv('/Users/brandonrose/Downloads/S1_File/Data_2.csv')
df3 = pd.read_csv('/Users/brandonrose/Downloads/S1_File/Data_3.csv')
df4 = pd.read_csv('/Users/brandonrose/Downloads/S1_File/Data_4.csv')
df5 = pd.read_csv('/Users/brandonrose/Downloads/S1_File/Data_5.csv')

df = pd.DataFrame()
df = df.append([df1,df2,df3,df4,df5])

/var/folders/pp/vgfp1wf143q46m_8v6qbt3bw0000gn/T/ipykernel_42494/954246736.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([df1,df2,df3,df4,df5])


In [13]:
base_features = df.columns[:7]

Let's generate a regex to extract the year from each column

In [14]:
exp = re.compile(r'(19\d{2}|20\d{2})')

Next we generate lookups for each variable as well as a reverse lookup for features.

`variable_lookup` has the original column headers with their new mappings and the associated year.

`features` has the new feature names mapped back to the original column headers.

In [15]:
variable_lookup = {}
for i in df.columns[7:]:
    try:
        year = re.findall(exp, i)[0]
        variable_lookup[i] = (int(year), 
                              i.replace(f"{year}", '')\
                              .replace('..','.')\
                              .rstrip('.'))
    except:
        variable_lookup[i] = ('All', i)
    
features = {}
for kk, vv in variable_lookup.items():
    if vv[1] not in features:
        features[vv[1]]=[kk]
    else:
        features[vv[1]].append(kk)

Now let's filter down to just dyads with > 1000 refugees in a single year:

In [16]:
kk = 'immigrant.population'
vv = features[kk]

In [17]:
df_ = df[list(base_features) + vv]
df_ = pd.melt(df_, id_vars=base_features, value_vars=vv)

In [170]:
refugee_dyads = df_[df_['value']>1000]['dyad.id'].unique()

In [175]:
print(f"There are {len(refugee_dyads)} dyads with more than 1000 refugees")

There are 315 dyads with more than 1000 refugees


In [176]:
df = df[df['dyad.id'].isin(refugee_dyads)]

In [177]:
df.shape

(1590, 376)

Now we can process each feature into it's own pivoted file:

In [178]:
def featurize(row):
    res = variable_lookup[row.variable]
    row['year'] = res[0]
    row['feature'] = res[1]
    return row

def get_feats(df):
    df = df.apply(lambda row: featurize(row), axis=1)
    return df

In [179]:
def parallelize(data, func):
    data_split = np.array_split(data, partitions)
    pool = mp.Pool(cores)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

In [180]:
print(f"There are {len(features.keys())} features to process.\n\nThey are {list(features.keys())}")

There are 24 features to process.

They are ['trade', 'ref.flow', 'alliance.defense', 'riv.strategic', 'rivalry.strategic.positional', 'rivalry.strategic.spatial', 'rivalry.strategic.ideological', 'rivalry.strategic.interv', 'contiguity', 'contiguity.any', 'min.distance', 'immigrant.population', 'pts.gradient', 'gdppc.gradient', 'polyarchy.additive.gradient', 'remittances', 'arms', 'remit', 'trips', 'arms.inverse', 'riv.strategic.positional', 'riv.strategic.ideological', 'riv.strategic.spatial', 'riv.strategic.interv']


In [181]:
for kk, vv in features.items():
    print(f"Trying {kk}")
    df_ = df[list(base_features) + vv]
    df_ = pd.melt(df_, id_vars=base_features, value_vars=vv)
    data = parallelize(df_, get_feats)
    data.rename(columns={'value': kk}, inplace=True)
    data.drop(labels=['variable','feature'], axis=1, inplace=True)
    data.to_csv(f'../refugee_data/s1_cleaned/{kk}.csv',index=False)
    del(data)
    del(df_)

Trying trade
Trying ref.flow
Trying alliance.defense
Trying riv.strategic
Trying rivalry.strategic.positional
Trying rivalry.strategic.spatial
Trying rivalry.strategic.ideological
Trying rivalry.strategic.interv
Trying contiguity
Trying contiguity.any
Trying min.distance
Trying immigrant.population
Trying pts.gradient
Trying gdppc.gradient
Trying polyarchy.additive.gradient
Trying remittances
Trying arms
Trying remit
Trying trips
Trying arms.inverse
Trying riv.strategic.positional
Trying riv.strategic.ideological
Trying riv.strategic.spatial
Trying riv.strategic.interv


Let's read back in those files and merge them back together

In [234]:
files = glob('../refugee_data/s1_cleaned/*.csv')
d = pd.read_csv(files[0]).dropna()

In [235]:
for file in files[1:]:
    print(file)
    d_ = pd.read_csv(file).dropna()
    d = pd.merge(d, d_.replace('All',1990), left_on=list(base_features)+['year'], 
                 right_on=list(base_features)+['year'], how='outer')

../refugee_data/s1_cleaned/rivalry.strategic.positional.csv
../refugee_data/s1_cleaned/pts.gradient.csv
../refugee_data/s1_cleaned/riv.strategic.positional.csv
../refugee_data/s1_cleaned/trade.csv
../refugee_data/s1_cleaned/polyarchy.additive.gradient.csv
../refugee_data/s1_cleaned/alliance.defense.csv
../refugee_data/s1_cleaned/arms.inverse.csv
../refugee_data/s1_cleaned/immigrant.population.csv
../refugee_data/s1_cleaned/gdppc.gradient.csv
../refugee_data/s1_cleaned/ref.flow.csv
../refugee_data/s1_cleaned/riv.strategic.spatial.csv
../refugee_data/s1_cleaned/arms.csv
../refugee_data/s1_cleaned/rivalry.strategic.ideological.csv
../refugee_data/s1_cleaned/riv.strategic.csv
../refugee_data/s1_cleaned/remittances.csv
../refugee_data/s1_cleaned/contiguity.csv
../refugee_data/s1_cleaned/rivalry.strategic.interv.csv
../refugee_data/s1_cleaned/min.distance.csv
../refugee_data/s1_cleaned/remit.csv
../refugee_data/s1_cleaned/trips.csv
../refugee_data/s1_cleaned/rivalry.strategic.spatial.csv
../

In [236]:
d.shape

(9540, 32)

In [237]:
d.head()

,ccode1,ccode2,state.destination.name,state.destination.abb,state.origin.name,state.origin.abb,dyad.id,riv.strategic.interv,year,rivalry.strategic.positional,...,riv.strategic,remittances,contiguity,rivalry.strategic.interv,min.distance,remit,trips,rivalry.strategic.spatial,riv.strategic.ideological,contiguity.any
0,2,20,Canada,CAN,United States of America,USA,220,0.0,2016,NaN,...,0.0,NaN,NaN,NaN,NaN,881.0,0.0,NaN,0.0,NaN
1,2,255,Germany,GMY,United States of America,USA,2255,0.0,2016,NaN,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
2,20,255,Germany,GMY,Canada,CAN,20255,0.0,2016,NaN,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
3,135,2,United States of America,USA,Peru,PER,1352,0.0,2016,NaN,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
4,210,255,Germany,GMY,Netherlands,NTH,210255,0.0,2016,NaN,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN


Demonstrate that we have at least _some_ non nulls across a handful of fields

In [239]:
d[(d['trade'].notnull()) & \
  (d['ref.flow'].notnull()) & \
  (d['remit'].notnull()) & \
  (d['trips'].notnull()) & \
  (d['arms.inverse'].notnull())] \
    [list(base_features) + ['trade','ref.flow','remit','trips','arms.inverse']].head()

,ccode1,ccode2,state.destination.name,state.destination.abb,state.origin.name,state.origin.abb,dyad.id,trade,ref.flow,remit,trips,arms.inverse
636,2,20,Canada,CAN,United States of America,USA,220,346062.5900,236.0,0.0,0.0,2.40
637,2,255,Germany,GMY,United States of America,USA,2255,123181.0400,0.0,0.0,0.0,0.00
638,20,255,Germany,GMY,Canada,CAN,20255,15889.1590,0.0,0.0,0.0,0.00
639,135,2,United States of America,USA,Peru,PER,1352,9759.9238,0.0,0.0,0.0,1.28
640,210,255,Germany,GMY,Netherlands,NTH,210255,84728.8980,1.0,0.0,8131865.0,0.00


Fix issues with `contiguity`, `contiguity.any` and `min.distance` since those are fixed over time. We set them to null except for 1990 so let's fill all other years with the values from 1990. 

In [240]:
def impute_dist_cont(row):
    if row.year==1990:
        return row
    else:
        imputer = d[(d['year']==1990)&(d['ccode1']==row.ccode1)&(d['ccode2']==row.ccode2)]
        row['contiguity.any'] = imputer['contiguity.any'].iloc[0]
        row['contiguity'] = imputer['contiguity'].iloc[0]
        row['min.distance'] = imputer['min.distance'].iloc[0]
        return row

In [241]:
d = d.apply(lambda row: impute_dist_cont(row), axis=1)

Let's check that this worked.

In [242]:
d['contiguity'].unique()

array([1., 0., 2., 4., 3., 5.])

In [243]:
d.groupby(['year','min.distance']).ccode1.count()

year  min.distance
1960  0.000000e+00    30
      1.410000e-12     2
      1.520000e-12     1
      1.760000e-12     1
      1.860000e-12     1
                      ..
2016  9.896475e+03     1
      9.911236e+03     1
      1.030244e+04     1
      1.054190e+04     1
      1.223704e+04     2
Name: ccode1, Length: 6420, dtype: int64

In [245]:
d.to_csv('../refugee_data/s1_process.csv', index=False)

## Analyzing processed graph data

In [61]:
df = pd.read_csv('../refugee_data/s1_process.csv')
df.head()

,ccode1,ccode2,state.destination.name,state.destination.abb,state.origin.name,state.origin.abb,dyad.id,riv.strategic.interv,year,rivalry.strategic.positional,...,riv.strategic,remittances,contiguity,rivalry.strategic.interv,min.distance,remit,trips,rivalry.strategic.spatial,riv.strategic.ideological,contiguity.any
0,2,20,Canada,CAN,United States of America,USA,220,0.0,2016,NaN,...,0.0,NaN,1.0,NaN,0.000000,881.0,0.0,NaN,0.0,1.0
1,2,255,Germany,GMY,United States of America,USA,2255,0.0,2016,NaN,...,0.0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0
2,20,255,Germany,GMY,Canada,CAN,20255,0.0,2016,NaN,...,0.0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0
3,135,2,United States of America,USA,Peru,PER,1352,0.0,2016,NaN,...,0.0,NaN,0.0,NaN,2845.444876,0.0,0.0,NaN,0.0,0.0
4,210,255,Germany,GMY,Netherlands,NTH,210255,0.0,2016,NaN,...,0.0,NaN,1.0,NaN,NaN,0.0,0.0,NaN,0.0,1.0


In [63]:
df.columns

Index(['ccode1', 'ccode2', 'state.destination.name', 'state.destination.abb',
       'state.origin.name', 'state.origin.abb', 'dyad.id',
       'riv.strategic.interv', 'year', 'rivalry.strategic.positional',
       'pts.gradient', 'riv.strategic.positional', 'trade',
       'polyarchy.additive.gradient', 'alliance.defense', 'arms.inverse',
       'immigrant.population', 'gdppc.gradient', 'ref.flow',
       'riv.strategic.spatial', 'arms', 'rivalry.strategic.ideological',
       'riv.strategic', 'remittances', 'contiguity',
       'rivalry.strategic.interv', 'min.distance', 'remit', 'trips',
       'rivalry.strategic.spatial', 'riv.strategic.ideological',
       'contiguity.any'],
      dtype='object')

In [12]:
ukr = df[df['state.origin.abb']=='UKR']
ukr.head()

,ccode1,ccode2,state.destination.name,state.destination.abb,state.origin.name,state.origin.abb,dyad.id,riv.strategic.interv,year,rivalry.strategic.positional,...,riv.strategic,remittances,contiguity,rivalry.strategic.interv,min.distance,remit,trips,rivalry.strategic.spatial,riv.strategic.ideological,contiguity.any
13,369,2,United States of America,USA,Ukraine,UKR,3692,0.0,2016,NaN,...,0.0,NaN,0.0,NaN,6.212676e+03,0.0,0.0,NaN,0.0,0.0
14,369,365,Russia,RUS,Ukraine,UKR,369365,0.0,2016,NaN,...,0.0,NaN,1.0,NaN,2.830000e-12,0.0,0.0,NaN,0.0,1.0
15,369,370,Belarus,BLR,Ukraine,UKR,369370,0.0,2016,NaN,...,0.0,NaN,1.0,NaN,0.000000e+00,0.0,0.0,NaN,0.0,1.0
101,369,210,Netherlands,NTH,Ukraine,UKR,369210,0.0,2016,NaN,...,0.0,NaN,0.0,NaN,1.127019e+03,0.0,0.0,NaN,0.0,0.0
102,369,211,Belgium,BEL,Ukraine,UKR,369211,0.0,2016,NaN,...,0.0,NaN,0.0,NaN,1.161451e+03,0.0,0.0,NaN,0.0,0.0


In [38]:
df_[df_['state.destination.name']=='Moldova']

,ccode1,ccode2,state.destination.name,state.destination.abb,state.origin.name,state.origin.abb,dyad.id,variable,value
116,2,359,Moldova,MLD,United States of America,USA,2359,ref2016.flow,0.0
295,20,359,Moldova,MLD,Canada,CAN,20359,ref2016.flow,0.0
445,31,359,Moldova,MLD,Bahamas,BHM,31359,ref2016.flow,0.0
648,40,359,Moldova,MLD,Cuba,CUB,40359,ref2016.flow,0.0
837,41,359,Moldova,MLD,Haiti,HAI,41359,ref2016.flow,0.0
...,...,...,...,...,...,...,...,...,...
5105216,223,359,Moldova,MLD,Liechtenstein,LIE,223359,ref2009.flow,NaN
5106452,626,359,Moldova,MLD,South Sudan,SSD,626359,ref2009.flow,NaN
5106486,835,359,Moldova,MLD,Brunei,BRU,835359,ref2009.flow,NaN
5106698,970,359,Moldova,MLD,Nauru,NAU,970359,ref2009.flow,NaN


In [22]:
ukr['state.destination.name'].unique()

array(['United States of America', 'Ecuador', 'Poland', 'Hungary',
       'Slovakia', 'Croatia', 'Greece', 'Cyprus', 'Russia', 'Estonia',
       'Belarus', 'Armenia', 'Georgia', 'Jordan', 'Canada', 'Mexico',
       'Costa Rica', 'Peru', 'Brazil', 'Argentina', 'Uruguay', 'Ireland',
       'Netherlands', 'Belgium', 'Luxembourg', 'France', 'Switzerland',
       'Portugal', 'Germany', 'Austria', 'Czech Republic', 'Italy',
       'Malta', 'Slovenia', 'Moldova', 'Romania', 'Lithuania', 'Finland',
       'Sweden', 'Norway', 'Denmark', 'Iceland', 'South Korea',
       'Australia', 'United Kingdom', 'Spain', 'Kyrgyzstan', 'Yugoslavia',
       'Venezuela', 'Bolivia', 'Macedonia', 'Azerbaijan',
       'Equatorial Guinea', 'Mali', 'Senegal', 'Niger', 'Guinea',
       'Nigeria', 'Central African Republic', 'Chad',
       'Democratic Republic of the Congo', 'Uganda', 'Kenya', 'Tanzania',
       'Burundi', 'Rwanda', 'Ethiopia', 'Eritrea', 'Angola', 'Mozambique',
       'Zambia', 'Zimbabwe', 'Namibia'

In [82]:
ukr_neighbors = ['POL','MLD','HUN','SLO','ROM']

In [71]:
ukr = df_[(df_['state.origin.abb']=='UKR')]

In [77]:
ukr['year'] = ukr.variable.apply(lambda x: x[-4:])

/var/folders/pp/vgfp1wf143q46m_8v6qbt3bw0000gn/T/ipykernel_26817/2226399579.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ukr['year'] = ukr.variable.apply(lambda x: x[-4:])


In [78]:
ukr

,ccode1,ccode2,state.destination.name,state.destination.abb,state.origin.name,state.origin.abb,dyad.id,variable,value,year
14162,369,2,United States of America,USA,Ukraine,UKR,3692,immigrant.population.1960,381955.0,1960
14163,369,130,Ecuador,ECU,Ukraine,UKR,369130,immigrant.population.1960,25.0,1960
14164,369,290,Poland,POL,Ukraine,UKR,369290,immigrant.population.1960,1177694.0,1960
14165,369,310,Hungary,HUN,Ukraine,UKR,369310,immigrant.population.1960,12019.0,1960
14166,369,317,Slovakia,SLO,Ukraine,UKR,369317,immigrant.population.1960,1879.0,1960
...,...,...,...,...,...,...,...,...,...,...
1132766,369,437,Ivory Coast,CDI,Ukraine,UKR,369437,immigrant.population.2010,NaN,2010
1133132,369,710,China,CHN,Ukraine,UKR,369710,immigrant.population.2010,NaN,2010
1133133,369,750,India,IND,Ukraine,UKR,369750,immigrant.population.2010,NaN,2010
1134374,369,223,Liechtenstein,LIE,Ukraine,UKR,369223,immigrant.population.2010,NaN,2010


In [57]:
pd.DataFrame(ukr.groupby(['year','state.destination.name'])['value'].sum()).to_csv('~/Desktop/historic_flows.csv')

In [84]:
pd.DataFrame(ukr[(ukr['value']>0)&(ukr['state.destination.abb'].isin(ukr_neighbors))]\
             .groupby(['year','state.destination.name'])['value'].sum())\
             .to_csv('~/Desktop/historic_migrant_stocks.csv')

In [20]:
df_.variable.unique()

array(['immigrant.population.1960', 'immigrant.population.1970',
       'immigrant.population.1980', 'immigrant.population.1990',
       'immigrant.population.2000', 'immigrant.population.2010'],
      dtype=object)

In [27]:
df_[(df_['variable']=='immigrant.population.2000') & (df_['state.origin.name']=='Ukraine')].sort_values(by='value', ascending=False)[:25]

,ccode1,ccode2,state.destination.name,state.destination.abb,state.origin.name,state.origin.abb,dyad.id,variable,value
770770,369,365,Russia,RUS,Ukraine,UKR,369365,immigrant.population.2000,3559975.0
770762,369,2,United States of America,USA,Ukraine,UKR,3692,immigrant.population.2000,444101.0
770847,369,705,Kazakhstan,KZK,Ukraine,UKR,369705,immigrant.population.2000,402671.0
770764,369,290,Poland,POL,Ukraine,UKR,369290,immigrant.population.2000,331462.0
770796,369,359,Moldova,MLD,Ukraine,UKR,369359,immigrant.population.2000,218596.0
770772,369,370,Belarus,BLR,Ukraine,UKR,369370,immigrant.population.2000,187293.0
770938,369,704,Uzbekistan,UZB,Ukraine,UKR,369704,immigrant.population.2000,162266.0
770790,369,255,Germany,GMY,Ukraine,UKR,369255,immigrant.population.2000,58163.0
770912,369,666,Israel,ISR,Ukraine,UKR,369666,immigrant.population.2000,56111.0
770776,369,20,Canada,CAN,Ukraine,UKR,36920,immigrant.population.2000,53801.0


In [25]:
df_['year'] = df_.variable.apply(lambda x: int(x[3:7]))

In [27]:
df_2016 = df_[df_['year']==2016]

ccode1 is the **origin** and ccode2 is the **destination**

In [46]:
df_2016[((df_2016['ccode1']==2)|(df_2016['ccode2']==2))&(df_2016['value']>0)].sort_values(by=['value'], ascending=False).head()

,ccode1,ccode2,state.destination.name,state.destination.abb,state.origin.name,state.origin.abb,dyad.id,variable,value,year
3712,92,2,United States of America,USA,El Salvador,SAL,922,ref2016.flow,3058.0,2016
3316,90,2,United States of America,USA,Guatemala,GUA,902,ref2016.flow,2464.0,2016
3492,91,2,United States of America,USA,Honduras,HON,912,ref2016.flow,2210.0,2016
2910,70,2,United States of America,USA,Mexico,MEX,702,ref2016.flow,1339.0,2016
27166,652,2,United States of America,USA,Syria,SYR,6522,ref2016.flow,793.0,2016


In [49]:
ccode_lookup = dict(zip(df_.ccode1,df['state.origin.name']))

In [36]:
df_2016[(df_2016['ccode1']==100)&(df_2016['ccode2']==130)]

,ccode1,ccode2,state.destination.name,state.destination.abb,state.origin.name,state.origin.abb,dyad.id,variable,value,year
4471,100,130,Ecuador,ECU,Colombia,COL,100130,ref2016.flow,7190.0,2016
38340,100,130,Ecuador,ECU,Colombia,COL,100130,ref2016.flow,NaN,2016
76170,100,130,Ecuador,ECU,Colombia,COL,100130,ref2016.flow,NaN,2016
114000,100,130,Ecuador,ECU,Colombia,COL,100130,ref2016.flow,NaN,2016
151830,100,130,Ecuador,ECU,Colombia,COL,100130,ref2016.flow,NaN,2016
